In [1]:
cd ..

c:\Users\Usuario\OneDrive\Documents\IE\3. Trimestre\Venture Lab & Capstone\Capstone\Tech side\vl_optimizer\equation_modeling


In [2]:
cd ..

c:\Users\Usuario\OneDrive\Documents\IE\3. Trimestre\Venture Lab & Capstone\Capstone\Tech side\vl_optimizer


Import required libraries

In [3]:
import pandas as pd
import numpy as np

# Chiller Group
## Code to generate the synthesized data required for the Chiller Group constraint formula:

Importing files

In [4]:
df = pd.read_csv('data/processed_data/industrial_sites_processed/2024-07-07T16-56-34_industrial_site2_processed.csv')

Choosing the right columns

In [5]:
columns_to_include = ['Time', '03 Chiller Group_Electric_Active Energy (kWh)', 'temp', 'humidity', 'heat_index', '03 Chiller Group_Electric_Power Factor (real)']
df_for_synthesis = df[columns_to_include]

Renaming columns

In [6]:
df_for_synthesis = df_for_synthesis.rename(columns={'temp': 'outdoor_temp', 'humidity': 'outdoor_humidity', '03 Chiller Group_Electric_Power Factor (real)': 'Chiller Group_Electric_Power Factor (real)(efficiency)'})
df_for_synthesis.to_csv('data/target_variables/target_chiller_group.csv')

Synthesized data generation

In [7]:
file_path = 'data/target_variables/target_chiller_group.csv'
data = pd.read_csv(file_path)

# Define the function to calculate chiller settings
def calculate_chiller_settings(heat_index):
    # Calculate chiller set point
    if heat_index < 18:
        set_point = 18
    elif heat_index > 40:
        set_point = 25
    else:
        set_point = round(18 + (heat_index - 18) * (7 / (40 - 18)))
    
    # Calculate number of active chillers
    if heat_index < 18:
        active_chillers = 0
    else:
        # Simple model: the number of chillers increases with heat_index
        # This is a simplistic linear relationship for demonstration purposes
        heat_index_factor = (heat_index - 18) / (40 - 18)
        active_chillers = int(np.ceil(heat_index_factor * 10))  # Example scale factor, adjust as needed
        
    return set_point, active_chillers

# Assuming your existing DataFrame is named df and has columns 'outdoor_temp' and 'humidity'
# Apply the function to each row in the DataFrame
data[['set_point', 'num_active_chiller']] = data.apply(
    lambda row: pd.Series(calculate_chiller_settings(row['heat_index'])),
    axis=1
)

data.head()

,Unnamed: 0,Time,03 Chiller Group_Electric_Active Energy (kWh),outdoor_temp,outdoor_humidity,heat_index,Chiller Group_Electric_Power Factor (real)(efficiency),set_point,num_active_chiller
0,0,2022-07-01 00:00:00,195.50,15.97,0.37,15.97,0.78375,18,0
1,1,2022-07-01 01:00:00,195.25,15.10,0.37,15.10,0.77975,18,0
2,2,2022-07-01 02:00:00,162.25,14.80,0.38,14.80,0.79450,18,0
3,3,2022-07-01 03:00:00,157.50,15.10,0.38,15.10,0.78075,18,0
4,4,2022-07-01 04:00:00,158.25,15.97,0.38,15.97,0.78475,18,0


Droping unwanted columns

In [8]:
data = data.drop(columns=['Unnamed: 0'])

Saving synthesized data

In [9]:
data.to_csv('data/synthesized_data/chiller_group_synthesized.csv', index=False)